# Task 3
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [19]:
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import os
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision.models import resnet152, ResNet152_Weights
from sklearn.model_selection import KFold
from torch.optim import Adam
# Add any other imports you need here
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# The device is automatically set to GPU if available, otherwise CPU
# If you want to force the device to CPU, you can change the line to
# device = torch.device("cpu")
# When using the GPU, it is important that your model and all data are on the
# same device.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [21]:
# Global Variables
global_batch_size = 50
global_num_workers = 8
#global_model = models.resnet18(weights='IMAGENET1K_V1')
global_model = resnet152(weights=ResNet152_Weights.IMAGENET1K_V2)
global_embedding_size = 512

In [ ]:
"""
Transform, resize and normalize the images and then use a pretrained model to extract
the embeddings.
"""
# TODO: define a transform to pre-process the images
# The required pre-processing depends on the pre-trained model you choose
# below.
# See https://pytorch.org/vision/stable/models.html#using-the-pre-trained-models
train_transforms = ResNet152_Weights.IMAGENET1K_V2.transforms

train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/dataset/", transform=train_transforms)
# Hint: adjust batch_size and num_workers to your PC configuration, so that you don't
# run out of memory (VRAM if on GPU, RAM if on CPU)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=32,
                          shuffle=False,
                          pin_memory=True,
                          num_workers=4)

# TODO: define a model for extraction of the embeddings (Hint: load a pretrained model,
# more info here: https://pytorch.org/vision/stable/models.html)
model = global_model
model.eval()
model.to(device)

In [37]:
def generate_embeddings():
    embedding_size = global_embedding_size # Dummy variable, replace with the actual embedding size once you
    # pick your model
    num_images = len(train_dataset)
    embeddings = np.zeros((num_images, embedding_size))
    # TODO: Use the model to extract the embeddings. Hint: remove the last layers of the
    # model to access the embeddings the model generates.
    feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])

    with torch.no_grad():
        start_idx = 0
        for inputs, _ in train_loader:
            inputs = inputs.to(device)
            outputs = feature_extractor(inputs)
            print(outputs.shape)
            embed = outputs.resize_(outputs.shape[0], outputs.shape[1])
            print(embed.shape)
            batch_size = outputs.shape[0]
            embeddings[start_idx:start_idx+batch_size] = outputs
            start_idx += batch_size
            print(start_idx)
    np.save('/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/embeddings3.npy', embeddings)

In [38]:
def get_data(file, train=True):
    """
    Load the triplets from the file and generate the features and labels.

    input: file: string, the path to the file containing the triplets
          train: boolean, whether the data is for training or testing

    output: X: numpy array, the features
            y: numpy array, the labels
    """
    triplets = []
    with open(file) as f:
        for line in f:
            triplets.append(line)

    # generate training data from triplets
    train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/dataset/",
                                         transform=None)
    filenames = [s[0].split('/')[-1].replace('.jpg', '') for s in train_dataset.samples]
    embeddings = np.load('/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/embeddings.npy')
    # TODO: Normalize the embeddings
    embeddings /= np.linalg.norm(embeddings, axis=1, keepdims=True)
    file_to_embedding = {}
    for i in range(len(filenames)):
      file_to_embedding[filenames[i]] = embeddings[i]
    X = []
    y = []
    # use the individual embeddings to generate the features and labels for triplets
    for t in triplets:
        emb = [file_to_embedding[a] for a in t.split()]
        X.append(np.hstack([emb[0], emb[1], emb[2]]))
        y.append(1)
        # Generating negative samples (data augmentation)
        if train:
            X.append(np.hstack([emb[0], emb[2], emb[1]]))
            y.append(0)
    X = np.vstack(X)
    y = np.hstack(y)
    return X, y

Hint: adjust batch_size and num_workers to your PC configuration, so that you don't run out of memory (VRAM if on GPU, RAM if on CPU)

In [39]:
def create_loader_from_np(X, y=None, train=True, batch_size=64, shuffle=True, num_workers=8):
    if train:
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float),
                                torch.from_numpy(y).type(torch.long))
    else:
        if y is not None:
            dataset = TensorDataset(torch.from_numpy(X).type(torch.float),
                                    torch.from_numpy(y).type(torch.long))
        else:
          dataset = TensorDataset(torch.from_numpy(X).type(torch.float))
    loader = DataLoader(dataset=dataset,
                        batch_size=batch_size,
                        shuffle=shuffle,
                        pin_memory=True, num_workers=num_workers)
    return loader


TODO: define a model. Here, the basic structure is defined, but you need to fill in the details

In [41]:
class Net(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        """self.fc1 = nn.Linear(input_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 64)
        self.fc4 = nn.Linear(64, 1)"""
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc3(x))
        #x = F.relu(self.fc4(x))
        return x

In [43]:
def test_model(model, loader):
    model.eval()
    predictions = []
    # Iterate over the test data
    with torch.no_grad(): # We don't need to compute gradients for testing
        for [x_batch] in loader:
            x_batch= x_batch.to(device)
            predicted = model(x_batch)
            predicted = predicted.cpu().numpy()
            # Rounding the predictions to 0 or 1
            predicted[predicted >= 0.5] = 1
            predicted[predicted < 0.5] = 0
            predictions.append(predicted)
        predictions = np.vstack(predictions)
    np.savetxt("/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/results_task3_7.txt", predictions, fmt='%i')

In [45]:
def train_model(train_loader):
    """
    The training procedure of the model; it accepts the training data, defines the model
    and then trains it.

    input: train_loader: torch.data.util.DataLoader, the object containing the training data

    output: model: torch.nn.Module, the trained model
    """
    model = Net(3*512)
    model.train()
    model.to(device)
    n_epochs = 5
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # TODO: define a loss function, optimizer and proceed with training. Hint: use the part
    # of the training data as a validation split. After each epoch, compute the loss on the
    # validation split and print it out. This enables you to see how your model is performing
    # on the validation data before submitting the results on the server. After choosing the
    # best model, train it on the whole training data.
    for epoch in range(n_epochs):
        running_loss = 0.0
        for [X, y] in train_loader:
          inputs = X.to(device)
          labels = y.to(device)

          # zero the parameter gradients
          optimizer.zero_grad()
          # forward
          # track history if only in train
          with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            # backward + optimize only if in training phase
            loss.backward()
            optimizer.step()
          running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Loss: {epoch_loss:.4f}')  # Acc: {epoch_acc:.4f}')
    return model

In [ ]:
# load the training data
TRAIN_TRIPLETS = '/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/train_triplets.txt'
TEST_TRIPLETS = '/content/drive/MyDrive/Colab Notebooks/IML FS24/task3/test_triplets.txt'

X, y = get_data(TRAIN_TRIPLETS)
# Create data loaders for the training data
train_loader = create_loader_from_np(X, y, train = True, batch_size=64)
# delete the loaded training data to save memory, as the data loader copies
del X
del y

# repeat for testing data
X_test, y_test = get_data(TEST_TRIPLETS, train=False)
test_loader = create_loader_from_np(X_test, train = False, batch_size=2048, shuffle=False)
del X_test
del y_test

# define a model and train it
model = train_model(train_loader)

# test the model on the test data
test_model(model, test_loader)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Loss: 0.6932
Loss: 0.6931
Loss: 0.6931
Loss: 0.6931
